In [6]:
import subprocess
import time
import psutil
import os

In [7]:
def run_stress_ng(method, worker_num):
    command = f"stress-ng --cpu-method {method} --cpu {worker_num} --timeout 40s --metrics"
    print(command)
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, preexec_fn=os.setsid)
    return process

In [8]:
def get_worker_pids(parent_pid):
    parent_process = psutil.Process(parent_pid)
    child_processes = parent_process.children(recursive=True)
    running_child_processes = [child for child in child_processes if child.status() != psutil.STATUS_SLEEPING]
    print(running_child_processes)

    worker_pids = [child.pid for child in running_child_processes]
    return worker_pids

In [9]:
def run_top_with_pids(pids, output_file):
    pid_list = ",".join(str(pid) for pid in pids)
    command = f"top -b -p {pid_list} -n 20 -d 1"
    print(command)
    
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    with open(output_file, 'w') as file:
        file.write(result.stdout.decode())

In [10]:
method = "float80"
max_worker_num = 16
for i in range(max_worker_num):
    print(i + 1)
    stress_process = run_stress_ng(method, i+1)

    time.sleep(5)

    worker_pids = get_worker_pids(stress_process.pid)
    run_top_with_pids(worker_pids, f"{method}-out{i+1}.txt")
    
    time.sleep(20)

1
stress-ng --cpu-method float80 --cpu 1 --timeout 40s --metrics
[psutil.Process(pid=3388, name='stress-ng', status='running', started='13:50:52')]
top -b -p 3388 -n 20 -d 1
2
stress-ng --cpu-method float80 --cpu 2 --timeout 40s --metrics
[psutil.Process(pid=3433, name='stress-ng', status='running', started='13:51:36'), psutil.Process(pid=3434, name='stress-ng', status='running', started='13:51:36')]
top -b -p 3433,3434 -n 20 -d 1
3
stress-ng --cpu-method float80 --cpu 3 --timeout 40s --metrics
[psutil.Process(pid=3452, name='stress-ng', status='running', started='13:52:20'), psutil.Process(pid=3453, name='stress-ng', status='running', started='13:52:20'), psutil.Process(pid=3454, name='stress-ng', status='running', started='13:52:20')]
top -b -p 3452,3453,3454 -n 20 -d 1
4
stress-ng --cpu-method float80 --cpu 4 --timeout 40s --metrics
[psutil.Process(pid=3462, name='stress-ng', status='running', started='13:53:04'), psutil.Process(pid=3463, name='stress-ng', status='running', started=